# **How to visualize your xml-model via code?**

## **Example model: 3-joint planar manipulator**
```xml
<mujoco model="3-joint-planar-manipulator">

    <default>
        <joint axis="0 1 0"/>
        <geom type="capsule" size=".01" rgba=".9 .7 .1 1"/>
        <default class="visual_cylinder">
            <geom type="cylinder" fromto="0 .015 0 0 -.015 0" size=".02" rgba=".3 .9 .3 .4"/>
        </default>
    </default>

    <worldbody>
        <light pos="0 0 1"/>
        <body pos="0 0 0">
            <joint/>
            <geom name="g1" class="visual_cylinder"/>
            <geom fromto="0 0 0 .1 0 0"/>
            <body pos=".1 0 0">
                <joint/>
                <geom name="g2" class="visual_cylinder"/>
                <geom fromto="0 0 0 .1 0 0"/>
                <body pos=".1 0 0">
                    <joint/>
                    <geom name="g3" class="visual_cylinder"/>
                    <geom fromto="0 0 0 .1 0 0"/>
                </body>
            </body>
        </body>
    </worldbody>

</mujoco>
```

## **Python example**

```python
import time
import mujoco
import mujoco.viewer

model = mujoco.MjModel.from_xml_path('example.xml')
data = mujoco.MjData(model)

with mujoco.viewer.launch_passive(model, data) as viewer:
    # Close the viewer automatically after 30 wall-seconds.
    start = time.time()
    while viewer.is_running() and time.time() - start < 30:
        step_start = time.time()

        # mj_step can be replaced with code that also evaluates
        # a policy and applies a control signal before stepping the physics.
        mujoco.mj_step(model, data)

        # Pick up changes to the physics state, apply perturbations, update options from GUI.
        viewer.sync()

        # Rudimentary time keeping, will drift relative to wall clock.
        time_until_next_step = model.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)
```

# **How dynamical simulators work?**

MuJoCo computes both forward and inverse dynamics in continuous time. The general equations of motion in continuous time are:
$$
M(q)\ddot{x} + c(q,\dot{x}) = \tau + J_c(q)^Tf(q, \dot{x}, \tau)
$$

Forward dynamics:
$$
\ddot{x} = M^{-1}(\tau + J_c^Tf - c)
$$

Inverse dynamics:
$$
\tau = M\ddot{x} + c - J_c^Tf
$$

For more information, see here: https://mujoco.readthedocs.io/en/stable/computation/index.html

# **Mujoco `model` and `data`**


`mjModel` - https://mujoco.readthedocs.io/en/stable/APIreference/APItypes.html#mjmodel


`mjData` - https://mujoco.readthedocs.io/en/stable/APIreference/APItypes.html#mjdata

# **Task**
> Write a script that tries all possible configurations of an arbitrary robot, calculates the inverse dynamics excluding collisions, and saves all the results of the experiment - motor angles and torques to a .csv file.

> Make a [violinplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.violinplot.html) showing the distribution of torques in the different joints. x-axis - joint name, y-axis - torque.